In [ ]:
#dependencies
import pandas as pd
import numpy as np

In [ ]:
#csv path 
all_states_history_path = "data/all-states-history.csv"
all_covid_data_df = pd.read_csv(all_states_history_path)
all_covid_data_df.head()


In [ ]:
#creating a dictionary for making a shorter covid df with only relevant cols
covid_cols_dict = {
    'date': [x for x in all_covid_data_df['date']],
    'state': [x for x in all_covid_data_df['state']],
    'positive_cases': [x for x in all_covid_data_df['positive']],
    'total_tests': [x for x in all_covid_data_df['totalTestResults']]
}

In [ ]:
#creating another df 
covid_cols_df = pd.DataFrame(covid_cols_dict)
covid_cols_df

In [ ]:
#new column, percentage of positive cases
covid_cols_df['percentage_positive'] = (covid_cols_df['positive_cases']/covid_cols_df['total_tests']) * 100
covid_cols_df.head()

In [ ]:
#resetting index as the dates
covid_cols_df = covid_cols_df.set_index('date')


In [ ]:
covid_cols_df.head()

In [ ]:
#reading in the state abbreviations
path = '../perry/statesAbbrev.csv'
statesAbbrev_df = pd.read_csv(path)
statesAbbrev_df.sample(12)

In [ ]:
#list for state codes
state_codes = [x for x in statesAbbrev_df['Code']]

In [ ]:
#list for state names
state_names = [x for x in statesAbbrev_df['State']]

In [ ]:
#finding the codes for each of the states concerned here
state_codes[11], state_codes[12], state_codes[13], state_codes[14], state_codes[15], state_codes[16], state_codes[17], state_codes[18], state_codes[19], state_codes[20]

In [ ]:
#reading in the stay at home order df
path = '../perry/full_SAH_orders.csv'
sah_data_df = pd.read_csv(path)
sah_data_df.head()

In [ ]:
#creating a df dict to hold the state dfs, reading in the dfs
states_df_dict = {}
for x in range(11, 21):
    path = f'../sarah/google_trends/google_trends_csvs/state_csvs/US-{state_codes[x]}_cat_data.csv'
    states_df_dict[f'{state_codes[x]}_data_df'] = pd.read_csv(path)


In [ ]:
#testing to see how many states lie in the df dict
len(states_df_dict)

In [ ]:
#inspecting the keys (ie df names)
states_df_dict.keys()
states_df_dict.keys()

In [ ]:
#test printing a df
print(state_codes[11])
states_df_dict[f'{state_codes[11]}_data_df'].head(10)

In [ ]:
dates_list = [x for x in states_df_dict[f'{state_codes[11]}_data_df']['time']]
dates_list

In [ ]:
#testing the char indices for only the year-month
dates_list[0][0:7]

In [ ]:
#creating a list of unique year-month combos
all_month_year = [x[0:7] for x in dates_list]
month_year = set(all_month_year)
month_year = list(month_year)

In [ ]:
#function for appending dates per month, and their indices, to a month_date_dict
months_dates_dict = {}
def date_dict_append(m):
    month_date_list = []
    for x in range(0, len(dates_list)):
        if dates_list[x][0:7] == month_year[m]:
            month_date_list.append((dates_list[x], x))
    #months_dates_dict = {}
    months_dates_dict[f'{month_year[m]}'] = month_date_list

In [ ]:
#calling the function for each month in the month-year dict
for m in range(0, len(month_year)):
    date_dict_append(m)

In [ ]:
#printing the dict
print(months_dates_dict)

In [ ]:
#defining function for appending the covid data to each state df
def covid_data_append(state_index):
    global state_date_list
    state_date_list = []
    for date in dates_list:
        try:
            state_date_list.append(covid_cols_df[covid_cols_df['state'] == state_codes[state_index]].loc[date, 'positive_cases'])
        except KeyError:
            print(date)
            state_date_list.append('n/a')
            continue  
    states_df_dict[f'{state_codes[state_index]}_data_df']['covid_cases'] = state_date_list
    print(states_df_dict[f'{state_codes[state_index]}_data_df'].tail(20))

In [ ]:
#calling funct for states dfs index 40-50
for x in range(11, 20):
    covid_data_append(x)

In [ ]:
#defining a funct for appending whether stay at home orders or gathering bans are happening
def value_changer(index_no):
    print(sah_data_df[sah_data_df['state_name'] == state_names[index_no]]) 
    query = input('is there a stay at home order? Yes/No')
    if query == 'Yes':
        start_month = input('start month')
        end_month = input('end month')
        print(months_dates_dict[f'2020-{start_month}'])
        print(months_dates_dict[f'2020-{end_month}'])
        val1 = int(input('start index'))
        val2 = int(input('end index(first index after the end of the ban)'))
        response = input('gathering ban? Yes/No')
        if response == 'Yes':
            for x in range(val1, val2):
                states_df_dict[f'{state_codes[index_no]}_data_df'].iat[x, -2] = 'True'
                states_df_dict[f'{state_codes[index_no]}_data_df'].iat[x, -1] = 'True'
        elif response == 'No':
            for x in range(val1, val2):
                states_df_dict[f'{state_codes[index_no]}_data_df'].iat[x, -2] = 'True'
        print(states_df_dict[f'{state_codes[index_no]}_data_df'].iloc[(val1-2):(val2+2), :])
    elif query == 'No':
        print(f'{state_codes[index_no]}_data_df has no bans')
        print(states_df_dict[f'{state_codes[index_no]}_data_df'].tail(10))
    else:
        ask = input('Did you make a mistake?Yes/No')
        if ask == 'Yes':
            value_changer(index_no)
        else:
            print(f'{state_codes[index_no]}_data_df has no bans')
            print(states_df_dict[f'{state_codes[index_no]}_data_df'].tail(10))

In [ ]:
#calling the funct for all of the states ind 11-20
for x in range(11, 20):
    value_changer(x)

In [ ]:
#outputting the dfs back out
for x in range(11,20):
    output_path = f'../sarah/google_trends/google_trends_csvs/state_csvs/US-{state_codes[x]}_cat_data.csv'
    states_df_dict[f'{state_codes[x]}_data_df'].to_csv(output_path, index=False)

In [ ]:
#test re-reading back in
path = f'../google_trends/google_trends_csvs/state_csvs/US-WV_cat_data.csv'
test_WV_df = pd.read_csv(path)
test_WV_df.tail(20)

In [ ]:
#outputting the dfs back out
for x in range(11,20):

    output_path = f'../keiko/states_data/US-{state_codes[x]}_cat_data.csv'
    states_df_dict[f'{state_codes[x]}_data_df'].to_csv(output_path, index=False)